In [3]:
%matplotlib inline
import numpy as np
import os
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

from nuscenes.nuscenes import NuScenes
import nuscenes.lidarseg
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
import utils_obj_det
from utils_obj_det import get_radar_in_img, vis_obj_det_speed
import utils_data_viewer
from utils_data_viewer import radar_from_file, fuse_radars_in_ego, visualize_radar_in_img


import time
from scipy.spatial.transform import Rotation
from pyquaternion import Quaternion

import cv2
#import open3d as o3d


nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)
utils_obj_det.nusc = nusc
utils_data_viewer.nusc = nusc
#nusc.list_scenes()
sc_id = 4

scene = nusc.scene[sc_id]
log = nusc.get('log', scene['log_token'])
map = nusc.get('map', log['map_token'])
map_file = os.path.join(nusc.dataroot, map['filename'])

first_sample = nusc.get('sample', scene['first_sample_token'])
last_sample = nusc.get('sample', scene['last_sample_token'])
sample = first_sample

	
COCO_INSTANCE_CATEGORY_NAMES = ['__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']



Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.466 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [4]:
#Show Radar extracted speed for objects in camera view
vis_obj_det_speed(scene, "CAM_FRONT", record=True)

### Test Visualization

In [5]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
model.to('cuda')

cam = nusc.get('sample_data', sample['data']['CAM_FRONT'])
cam_file = os.path.join(nusc.dataroot, cam['filename'])
cam_img = cv2.imread(cam_file)
#Change to RGB only for conversion to tensor
cam_tensor = F.to_tensor(cv2.cvtColor(cam_img, cv2.COLOR_BGR2RGB)).unsqueeze(0).to('cuda')
# Perform inference
with torch.no_grad():
    pred = model(cam_tensor)

threshold = 0.9


pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())]
pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].cpu().detach().numpy().astype(int))]
pred_score = list(pred[0]['scores'].cpu().detach().numpy())
#Find last index which belongs to score bigger than threshold
pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
pred_boxes = pred_boxes[:pred_t+1]
pred_class = pred_class[:pred_t+1]
results = np.copy(cam_img)

rect_th=1
text_size=1
text_th=1


#Draw bounding boxes
for i in range(len(pred_boxes)):
    cv2.rectangle(results, pred_boxes[i][0], pred_boxes[i][1], color=(0, 255, 0), thickness=rect_th)
    cv2.putText(results, pred_class[i], pred_boxes[i][0], cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)

winname = cam['channel']
window_width, window_height = 800, 600
cv2.namedWindow(winname, cv2.WINDOW_NORMAL)
cv2.resizeWindow(winname, window_width, window_height)

cv2.imshow(winname, results)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
with torch.no_grad():
    pred = model(cam_tensor)